**Exploratory Data Analysis**

As pert of EDA, the following analysis needs to be performed:
1. Get maximum insights from a dataset
2. Uncover underlying structure
3. Extract important variables from the dataset
4. Detect outliers and anomalies
5. Test the underlying assumptions
6. Determine the underlying factor settings

In [2]:
# Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

In [ ]:
# Reading file
df=pd.read_csv('data.csv')
df

In [ ]:
df.columns

We can see that there are 8 columns namely event_time, order_id, product_id, category_id,category_code, brand, price and user_id.
Price will be the target column here.

In [ ]:
#Finding the data type of each column
df.dtypes

In [ ]:
# Finding null values in each column 
df.isnull().sum()

In [ ]:
len(df)

It can be see that 16% of category_id values, 23% of category_code values, 19% of brand values, 16% of price values and 78% of user_id values are missing.

In [ ]:
# Checking if the garbage data exists in individual column
for col in df.columns:
    print('{} : {}'.format(col,df[col].unique()))

No garbage value was observed. Empty columns already have nan value.

In [ ]:
# Visualising the missing values
sns.heatmap(df.isnull(),cbar=False,cmap='viridis')

We can see that the user_id column has too many missing values and we can drop this column since it's not relevant for price prediction.


In [ ]:
# Dropping column user_id
df=df.drop(['user_id'], axis=1)
df.head()

In [ ]:
# Replace numerical missing values with the mean value of the column
num_col = ['category_id',  'price']
for col in num_col:
    df[col]=pd.to_numeric(df[col])
    df[col].fillna(df[col].mean(), inplace=True)
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# Replace categorical values with mode of the column
cat_col=['category_code','brand']
for col in cat_col:    
    df[col].fillna(df[col].mode()[0], inplace = True)


In [ ]:
df.isnull().sum()

Now we see that we filled all the null values and we don't have any null values in the dataset

**Analysing data**

Once we have done the pre-processing, now let's analyse the relationship between different parameteres.
This can be achieved by finding the correlation factor between different parameteres.


In [ ]:
df.corr()

In [ ]:
len(df)

Since corr() function only returns the correlation between the numerical values, ignoring the categorical data, we need to encode the categorical data between finding the correlation.
Also the size of the dataset is huge. So using one-hot encoding will cost a lot of memory.


In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(),cbar=True,annot=True,cmap='Blues')

We see that price has a positive correlation with order_id and category_id and a negative correlation with product_id.
This is not the correct interpretation since id's are just unique identifiers of the object and shouldn't affect the price.

# Analyzing Data

**What is the maximum & median value of the order placed?**

In [ ]:
df_group_by = df.groupby(['order_id'], as_index=False)['price'].sum()
df_group_by

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
print(df_group_by[df_group_by['price'] == df_group_by['price'].max()])


In [ ]:
df_group_by.median()

In [ ]:
df[df['order_id']==2353234364520727464]

The maximum value of order placed is 52295.243 with order_id=2353234364520727464. In this order, 3 products were purchased. 2 products belonged to samsumg brand.

The median order value is 138.63

**Which product is most & least frequently bought?**




In [ ]:
df['product_id'].value_counts().sort_values()

In [ ]:
# product_id = 1515966223523303302 is the most frequent item being purchased
# product_id = 2388434452476057132,1515966223527522793,1515966223510683181,1515966223510235365 are among the least bought item 
df[df['product_id']==1515966223523303302]

In [ ]:
df[df['product_id']==1515966223510235365]


In [ ]:
df[df['product_id']==1515966223510683181]

product_id = 1515966223523303302 is the most frequent purchased product & product_id=1515966223510683181,1515966223510235365 are among the least purchased products.
One observation here is that the product which is frequently bought lies close to the median whereas the less frequent bought items are either too left or too right to the median.

**How many categories each product has**



In [ ]:
df_group_by = df.groupby(['product_id'])['category_id'].nunique()
df_group_by
# check if greater than 1

This shows that every product has 1 category_id associated to it. Every product belongs to just 1 category. Hence it's a one to one mapping

**Which product generates maximum revenue?**

In [ ]:
df_group_by = df.groupby(['product_id'], as_index=False)['price'].sum()
df_group_by.sort_values('price')
# ascending = False
# add head

In [ ]:
#product_id = 1515966223523303302 generates the maximum revenue
df[df['product_id']==1515966223523303302]

product_id=1515966223523303302 generates the maximum revenue. Also this is the most frequent bought product as we saw before.

**Which brand has maximum sale**

In [ ]:
# average revenue
df_group_by = df.groupby(['brand'], as_index=False)['price'].sum()
df_group_by.sort_values('price')

It can be observed that samsumg is the rand which generates the maximum revenue & index brand generates the minimum revenue.

**Algorithm Implementation**

In [3]:
df=pd.read_csv('data.csv')
df.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [4]:
num_col = ['category_id',  'price']
for col in num_col:
    df[col]=pd.to_numeric(df[col])
    df[col].fillna(df[col].mean(), inplace=True)

In [5]:
# Replace categorical values with mode of the column
cat_col=['category_code','brand']
for col in cat_col:    
    df[col].fillna(df[col].mode()[0], inplace = True)

In [6]:
df=df.sort_values('event_time')

In [7]:
df = df.dropna(subset=['user_id'])

In [8]:
df.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
42864,1970-01-01 00:33:40 UTC,2348531873749140190,1515966223509130920,2.268105e+18,computers.components.hdd,transcend,99.51,1.515916e+18
42883,1970-01-01 00:33:40 UTC,2348531875603022568,1515966223509106869,2.268105e+18,electronics.smartphone,xiaomi,220.78,1.515916e+18
42759,1970-01-01 00:33:40 UTC,2348531863666033329,1515966223509128873,2.268105e+18,electronics.audio.headphone,sony,9.24,1.515916e+18
43005,1970-01-01 00:33:40 UTC,2348531887120581416,1515966223509089509,2.268105e+18,electronics.smartphone,apple,416.41,1.515916e+18
42994,1970-01-01 00:33:40 UTC,2348531886130725667,1515966223509104202,2.268105e+18,computers.peripherals.monitor,samsung,358.77,1.515916e+18


In [19]:
item_user={}
for i in range(len(df)):
#     print(df['user_id'][i],df['category_code'][i])
    user_id=df['user_id'][i]
    category_code=df['category_code'][i]
#     print(user_id, category_code)
    try:
        item_user[user_id]=[category_code]
    except:
        item_user[user_id].append(category_code)
# print(item_user)

KeyError: 1812

In [22]:
df_group=df.groupby(['user_id'])['category_code'].apply(','.join).reset_index()

In [26]:
df_group

,user_id,category_code
0,1.515916e+18,electronics.video.tv
1,1.515916e+18,"kids.toys,computers.network.router"
2,1.515916e+18,"electronics.smartphone,electronics.video.tv,el..."
3,1.515916e+18,"electronics.smartphone,electronics.smartphone,..."
4,1.515916e+18,"appliances.kitchen.microwave,appliances.enviro..."
...,...,...
98257,1.515916e+18,electronics.smartphone
98258,1.515916e+18,computers.notebook
98259,1.515916e+18,"electronics.smartphone,electronics.smartphone,..."
98260,1.515916e+18,electronics.smartphone


In [63]:
DATASET=df_group['category_code']

In [104]:
dataset=[]
for i in range(len(DATASET)):
    dataset.append(DATASET[i].split(","))
#print(dataset)

/Users/siddharthshukla/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [68]:
from itertools import combinations

def generate_frequent_itemsets(dataset, min_support):
    itemsets = []
    support_counts = {}
    num_transactions = len(dataset)
    # Generate frequent 1-itemsets
    frequent_1_itemsets = {}
    for transaction in dataset:
        for item in transaction:
            
            itemset = frozenset([item])
            frequent_1_itemsets[itemset] = frequent_1_itemsets.get(itemset, 0) + 1
    
    frequent_itemsets = {1: frequent_1_itemsets}
    
    # Generate frequent k-itemsets (k > 1)
    k = 2
    while frequent_itemsets.get(k - 1):
        candidate_itemsets = set()
        
        # Generate candidate itemsets
        for itemset1 in frequent_itemsets[k - 1]:
            for itemset2 in frequent_itemsets[k - 1]:
                union_set = itemset1.union(itemset2)
                if len(union_set) == k:
                    candidate_itemsets.add(union_set)
        
        # Count support for candidate itemsets
        frequent_k_itemsets = {}
        for transaction in dataset:
            for itemset in candidate_itemsets:
                if itemset.issubset(transaction):
                    frequent_k_itemsets[itemset] = frequent_k_itemsets.get(itemset, 0) + 1
        
        # Prune infrequent itemsets
        frequent_k_itemsets = {itemset: count for itemset, count in frequent_k_itemsets.items()
                               if count / num_transactions >= min_support}
        
        if frequent_k_itemsets:
            frequent_itemsets[k] = frequent_k_itemsets
            itemsets.extend(frequent_k_itemsets.keys())
        
        k += 1
    return itemsets, frequent_itemsets


def generate_association_rules(frequent_itemsets, min_confidence):
    rules = []
    
    for k, itemsets in frequent_itemsets.items():
        if k < 2:
            continue
        
        for itemset in itemsets:
            for i in range(1, k):
                for antecedent in combinations(itemset, i):
                    antecedent = frozenset(antecedent)
                    consequent = itemset.difference(antecedent)
                    confidence = itemsets[itemset] / frequent_itemsets[len(antecedent)][antecedent]
                    
                    if confidence >= min_confidence:
                        rule = (antecedent, consequent, confidence)
                        rules.append(rule)
    
    return rules


In [75]:
min_support = 0.05
min_confidence = 0.05

itemsets, frequent_itemsets = generate_frequent_itemsets(dataset, min_support)
rules = generate_association_rules(frequent_itemsets, min_confidence)

# Print results
print("Frequent itemsets:")
for itemset in itemsets:
    print(itemset)

print("\nAssociation rules:")
for rule in rules:
    antecedent, consequent, confidence = rule
    print(f"{antecedent} => {consequent} (Confidence: {confidence})")



Frequent itemsets:
frozenset({'electronics.smartphone', 'electronics.video.tv'})
frozenset({'electronics.audio.headphone', 'electronics.smartphone'})
frozenset({'appliances.kitchen.refrigerators', 'electronics.smartphone'})
frozenset({'appliances.environment.vacuum', 'electronics.smartphone'})
frozenset({'computers.notebook', 'electronics.smartphone'})

Association rules:
frozenset({'electronics.video.tv'}) => frozenset({'electronics.smartphone'}) (Confidence: 0.32999320652173914)
frozenset({'electronics.audio.headphone'}) => frozenset({'electronics.smartphone'}) (Confidence: 0.33798827154358413)
frozenset({'appliances.kitchen.refrigerators'}) => frozenset({'electronics.smartphone'}) (Confidence: 0.3034166994879874)
frozenset({'appliances.environment.vacuum'}) => frozenset({'electronics.smartphone'}) (Confidence: 0.3167927004562215)
frozenset({'computers.notebook'}) => frozenset({'electronics.smartphone'}) (Confidence: 0.28796114259280675)


In [78]:
min_support = 0.06
min_confidence = 0.1

itemsets, frequent_itemsets = generate_frequent_itemsets(dataset, min_support)
rules = generate_association_rules(frequent_itemsets, min_confidence)

# Print results
print("Frequent itemsets:")
for itemset in itemsets:
    print(itemset)

print("\nAssociation rules:")
for rule in rules:
    antecedent, consequent, confidence = rule
    print(f"{antecedent} => {consequent} (Confidence: {confidence})")

Frequent itemsets:
frozenset({'electronics.audio.headphone', 'electronics.smartphone'})
frozenset({'appliances.kitchen.refrigerators', 'electronics.smartphone'})
frozenset({'computers.notebook', 'electronics.smartphone'})

Association rules:
frozenset({'electronics.audio.headphone'}) => frozenset({'electronics.smartphone'}) (Confidence: 0.33798827154358413)
frozenset({'appliances.kitchen.refrigerators'}) => frozenset({'electronics.smartphone'}) (Confidence: 0.3034166994879874)
frozenset({'computers.notebook'}) => frozenset({'electronics.smartphone'}) (Confidence: 0.28796114259280675)


In [123]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

te = TransactionEncoder()
te_array = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_array, columns=te.columns_)

from mlxtend.frequent_patterns import fpgrowth


frequent_itemsets_fp=fpgrowth(df, min_support=0.06, use_colnames=True)
rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=0.1)


/Users/siddharthshukla/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [124]:

frequent_itemsets_fp.to_excel('frequent_itemsets_fp_v1.xlsx')

/Users/siddharthshukla/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
